In [ ]:
# 主程式碼（不使用 if/else；僅用 try/except）

class DomainError(Exception):
    """領域錯誤：用來包裝較友善的訊息"""

# 1) 安全除法：不使用 if/else

def safe_divide(a, b):
    try:
        return a / b
    except ZeroDivisionError as e:
        raise DomainError("除數不可為零") from e
    except TypeError as e:
        raise DomainError("參數必須是數字") from e

# 2) 字串轉整數：不使用 if/else

def parse_int(text):
    try:
        return int(text)
    except ValueError as e:
        raise DomainError(f"無法轉為整數：{text}") from e

# 3) 取清單元素：不先檢查索引

def get_item(lst, index):
    try:
        return lst[index]
    except IndexError as e:
        raise DomainError(f"索引超出範圍：{index}") from e
    except TypeError as e:
        raise DomainError("提供的容器不支援索引或索引型別錯誤") from e

# 4) 取字典鍵：不先檢查鍵存在

def get_key(d, key):
    try:
        return d[key]
    except KeyError as e:
        raise DomainError(f"鍵不存在：{key}") from e
    except TypeError as e:
        raise DomainError("提供的物件不是字典或鍵型別錯誤") from e

# 5) 安全讀檔：直接 open，錯誤交給 except

def read_text_file(path, encoding="utf-8"):
    try:
        with open(path, "r", encoding=encoding) as f:
            return f.read()
    except FileNotFoundError as e:
        raise DomainError(f"找不到檔案：{path}") from e
    except PermissionError as e:
        raise DomainError(f"沒有權限讀取：{path}") from e
    except UnicodeDecodeError as e:
        raise DomainError("檔案編碼錯誤，請確認編碼") from e

# 6) API 呼叫（示意）：把第三方錯誤轉成 DomainError

import json
from urllib.request import urlopen
from urllib.error import URLError, HTTPError

def fetch_user(user_id):
    url = f"https://jsonplaceholder.typicode.com/users/{user_id}"
    try:
        with urlopen(url, timeout=5) as resp:
            return json.loads(resp.read().decode("utf-8"))
    except HTTPError as e:
        raise DomainError(f"HTTP 錯誤：{e.code}") from e
    except URLError as e:
        raise DomainError("網路連線失敗") from e
    except json.JSONDecodeError as e:
        raise DomainError("回應格式錯誤") from e

# 7) 範例：管線式資料處理

class Pipeline:
    def __init__(self, steps):
        self.steps = steps

    def run(self, value):
        current = value
        for step in self.steps:
            try:
                current = step(current)
            except Exception as e:
                raise DomainError(f"處理步驟 {step.__name__} 失敗") from e
        return current

# 小步驟

def to_int(x):
    return parse_int(x)

def inverse(x):
    return safe_divide(1, x)

def multiply_by_100(x):
    return x * 100

# 建立一條處理管線

pipeline = Pipeline([to_int, inverse, multiply_by_100])

# 測試（請於 Notebook 中呼叫並用 try/except 外層處理 DomainError）
# try:
#     print(pipeline.run("25"))    # 正常
#     print(pipeline.run("0"))     # 會在 inverse 失敗（除以零）
# except DomainError as e:
#     print("❌", e)



# 🧑‍💻 作業（請只用 try/except，不要使用 if/else）

以下每題都要求：
- 只能用 try/except（或 finally/with）來處理狀況，不可使用 if/else
- 發生錯誤時，請拋出 `DomainError`，並提供清楚、可讀的訊息
- 可以善用 `raise ... from e` 保留原始錯誤脈絡

---

## 題目 1：safe_reciprocal
實作函式 `safe_reciprocal(x)`，回傳 `1/x` 的結果。

需求：
- 除以 0 時，拋出 `DomainError("除數不可為零")`
- 型別錯誤（例如字串）時，拋出 `DomainError("參數必須是數字")`

提示：
- 直接回傳 `1 / x`，不要先檢查 x 是否為 0 或型別
- 捕獲 `ZeroDivisionError` 與 `TypeError` 後，轉拋 `DomainError`

---

## 題目 2：parse_json_field
實作函式 `parse_json_field(json_text, key)`，將字串 `json_text` 解析為物件，並回傳對應 `key` 的值。

需求：
- 解析失敗（格式不正確）時，拋出 `DomainError("回應格式錯誤")`
- 查無該鍵時，拋出 `DomainError(f"鍵不存在：{key}")`

提示：
- 使用 `json.loads(json_text)` 直接嘗試解析
- 存取 `obj[key]`，不要先檢查鍵是否存在
- 捕獲 `json.JSONDecodeError` 與 `KeyError` 後轉拋 `DomainError`

---

## 題目 3：copy_text_file
實作函式 `copy_text_file(src_path, dst_path, encoding="utf-8")`，將文字檔 `src_path` 複製到 `dst_path`。

需求：
- 檔案不存在時：`DomainError(f"找不到檔案：{src_path}")`
- 權限不足時：`DomainError("沒有權限讀取/寫入")`
- 編碼錯誤時：`DomainError("檔案編碼錯誤，請確認編碼")`
- 正確使用 `with` 語句確保資源釋放

提示：
- 用 `with open(src_path, 'r', encoding=encoding)` 讀取，再 `with open(dst_path, 'w', encoding=encoding)` 寫入
- 讓 `open` 與 `read()/write()` 自己丟錯；在 `except` 轉拋 `DomainError`

---

## 題目 4：管線擴充（Pipeline）
在既有 `Pipeline` 上，加上一個步驟 `as_percent(x)`，將數值轉換為百分比字串（例如 0.25 -> "25.00%"）。

需求：
- 輸入不是數字時，拋出 `DomainError("參數必須是數字")`
- 最終建立管線：`[to_int, inverse, multiply_by_100, as_percent]`

提示：
- `as_percent` 內直接嘗試格式化，例如：`f"{x:.2f}%"`
- 非數字導致的型別錯誤，捕獲 `TypeError` 或 `ValueError` 再拋出 `DomainError`

---

## 題目 5（進階）：整合外部資料
撰寫函式 `fetch_user_email(user_id)`：
- 呼叫既有的 `fetch_user(user_id)` 取得使用者 JSON
- 回傳該使用者的 `email` 欄位

需求：
- 網路錯誤、HTTP 錯誤、JSON 解析錯誤皆已由 `fetch_user` 轉成 `DomainError`，請在外層接住或讓其往上拋
- 查無 `email` 鍵時，拋出 `DomainError("鍵不存在：email")`

提示：
- 直接呼叫 `fetch_user` 取得 dict，然後存取 `user["email"]`
- 存取時若出錯，捕獲 `KeyError` 再拋出 `DomainError`

---

### 建議測試方式（僅示意）
```python
try:
    print(safe_reciprocal(4))     # 0.25
    print(safe_reciprocal(0))     # DomainError: 除數不可為零
except DomainError as e:
    print("❌", e)

try:
    print(parse_json_field('{"a": 1}', "a"))  # 1
    print(parse_json_field('{"a": 1}', "b"))  # DomainError: 鍵不存在：b
except DomainError as e:
    print("❌", e)
```

